# 학습된 모델 불러오기

In [ ]:
import pandas as pd
import torch
from transformers import PreTrainedTokenizerFast
from transformers.models.bart import BartForConditionalGeneration
import pandas as pd
from tqdm import tqdm

%cd /content/drive/MyDrive/파일위치/KoBART-summarization
model = BartForConditionalGeneration.from_pretrained('./kobart_summary')
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v1')

# 판결 요지 정리

In [ ]:
p = pd.read_pickle(("/content/drive/MyDrive/파일위치/pan_1_random.pkl"))
panyo_list = list(p['panyo']) #판결요지 리스트
avg = 0
d = {} #일련번호와 정리된 판결요지를 저장할 딕셔너리
#각 문단 별로 저장(각 문단의 평균 길이가 400보다 짧다면 분리하지 않고 저장)
for k in range(len(panyo_list)):
  pan = panyo_list[k]
  if len(pan) < 5:
    continue
  if len(pan) < 400: #길이가 400보다 짧으면 그대로 저장
    d[num_list[k]] = [pan]
    continue
  # [1] 또는 가. 또는 ① 형태로 번호가 매겨져 있지 않다면 <br/>을 기준으로 나누어 저장
  if pan.find('[1]') == -1 and pan.find('가.') == -1 and pan.find('①') == -1:
    if len(pan.split('<br/>')) == 1:
      continue
    s = pan.split('<br/>')
    total = 0
    for w in s:
        total += len(w)
    if total/len(s) < 400:
          d[num_list[k]] = [pan]
    else:
          d[num_list[k]] = s
  #각 문단이 ① 형태로 매겨져 있을 경우
  elif pan.find('①') != -1:
    i = 0
    s = pan.split('①')
    while True:
      try:
        s[i+1].index(chr(ord('①')+i+1)) #숫자를 하나씩 늘려가며 인덱스를 찾음
        s= [s[i]] + pan.split(chr(ord('①')+i+1)) #오류가 나지 않는다면 존재한다는 의미이므로 해당 기호로 분리
        i += 1
      except: #
        total = 0
        for w in s:
          total += len(w)
        if total/len(s) < 400:
          d[num_list[k]] = [pan]
        else:
          print(k)
          d[num_list[k]] = s
        break
  #각 문단이 가. 형태로 매겨져 있을 경우
  elif pan.find('가.') != -1:
    han_list=['가.', '나.', '다.', '라.', '마.', '바.', '사.'] #확인 결과 사. 이상인 것은 없었음
    s_list = pan.split('<br/>') #일차적으로 <br/>을 기준으로 나누기
    i = 0
    for _ in range(len(s_list)):
      s = s_list[i]
      if len(s) < 15:
        s_list.remove(s)
      else:
        s_list[i] = s.strip()
        i += 1
    for s in s_list:
      if len(s) < 15:
        continue
      c = 0
       #han_list 중 하나로 시작하는지 확인
      for h in han_list:
        if s.find(h) == 0:
          c = 1
          break
      if c == 0: #han_list의 요소로 시작하지 않을 경우
         #마지막 문단이면 위 문단과 합치고 아니라면 s_list를 비운다(예외 상황이 너무 많아서 그냥 없앰)
        if s_list.index(s) == len(s_list) -1:
          s_list.remove(s)
          s_list[len(s_list) -1] += s
        else:
          s_list = []
          break
    #S_list가 빈 리스트가 아니라면 d(전체 딕셔너리)에 추가
    if len(s_list) > 0:
      total = 0
      for w in s_list:
          total += len(w)
      if total/len(s_list) < 400:
          d[num_list[k]] = [pan]
      else:
          print(k)
          d[num_list[k]] = s_list
  #각 문단이 [1] 형태로 매겨져 있을 경우
  elif pan.find('[1]') != -1:
    i = 0
    s = pan.split('[1]')
    #하나씩 숫자를 높여가며 찾다가 없으면(오류가 나면) 저장 후 반복문 종료
    while True:
      try:
        s[i+1].index('[' + str(i) +']')
        s= [s[i]] + pan.split('[' + str(i) +']')
        i += 1
      except:
        total = 0
        for w in s:
          total += len(w)
        if total/len(s) < 400:
          d[num_list[k]] = [pan]
        else:
          d[num_list[k]] = s
        break
precedentDic = {}
precedentDic['number'] = list(d.keys())
precedentDic['contents'] = list(d.values())
p_obj = pd.DataFrame(precedentDic, columns = ['number', 'contents'])
p_obj.to_pickle("/content/drive/MyDrive/파일위치/panyo_1_random_8000_split.pkl")


# 요약

## 요약_ 조금씩 나눠서 진행

In [ ]:
from tqdm import tqdm
panD = {}
p_obj = pd.read_pickle('/content/drive/MyDrive/파일위치/panyo_1_random_8000_split.pkl' )
for d in tqdm(range(3000, len(p_obj))):
    output = ''
    for s in [p_obj['contents'][d]]: #각 문단 별로
      if len(s) < 5:
        continue
      elif len(s) < 50: #길이가 50보다 작으면 요약하지 않고 원문 저장
        panD[p_obj['number'][d]] = p_obj['contents'][d]
        continue
      input_ids = tokenizer.encode(s)
      if len(input_ids) > 1026: #이 경우 오랫동안 실행되다가 오류가 나서 바로 중단하도록 함
        break
      input_ids = torch.tensor(input_ids)
      input_ids = input_ids.unsqueeze(0)
      out = model.generate(input_ids, eos_token_id=1, max_length=1024, num_beams=5)
      out = tokenizer.decode(out[0], skip_special_tokens=True)
      output += out + '\n'
    panD[p_obj['number'][d]] = output
    if len(panD) != 0 and len(panD) % 100 == 0:
      precedentDic = {}
      precedentDic['number'] = list(panD.keys())
      precedentDic['contents'] = list(panD.values())
      p = pd.DataFrame(precedentDic, columns = ['number', 'contents'])
      p.to_pickle("/content/drive/MyDrive/파일위치/panyo_summary_1_random_8000_split(3).pkl")


## 데이터 프레임 합치기

In [ ]:
import pandas as pd
dList = []
df = []
dList.append(pd.read_pickle("/content/drive/MyDrive/파일위치/panyo_summary_1_random_8000_split.pkl"))
dList.append(pd.read_pickle("/content/drive/MyDrive/파일위치/panyo_summary_1_random_8000_split(2).pkl"))
dList.append(pd.read_pickle("/content/drive/MyDrive/파일위치/panyo_summary_1_random_8000_split(3).pkl"))
df = pd.concat(dList, sort=False)
df.index = range(len(df))
df.to_pickle("/content/drive/MyDrive/파일위치/panyo_summary_1_random_8000.pkl")


In [ ]:
import pandas as pd
dList = []
df = []
dList.append(pd.read_pickle("/content/drive/MyDrive/파일위치/panyo_summary_1_random_8000.pkl"))
dList.append(pd.read_pickle("/content/drive/MyDrive/파일위치/panyo_summary_2_random_8000.pkl"))
dList.append(pd.read_pickle("/content/drive/MyDrive/파일위치/panyo_summary_3_random_8000.pkl"))
dList.append(pd.read_pickle("/content/drive/MyDrive/파일위치/panyo_summary_4_random_8000.pkl"))
dList.append(pd.read_pickle("/content/drive/MyDrive/파일위치/panyo_summary_5_random_8000.pkl"))
dList.append(pd.read_pickle("/content/drive/MyDrive/파일위치/panyo_summary_6_random_8000.pkl"))
dList.append(pd.read_pickle("/content/drive/MyDrive/파일위치/panyo_summary_7_random_8000.pkl"))
df = pd.concat(dList, sort=False)
df.index = range(len(df))
df.to_pickle("/content/drive/MyDrive/파일위치/panyo_summary_total_random_8000.pkl")
